# Preprocess Payments Dataset

En este notebook, procesaremos el dataset de payments, este dataset incluye información importante sobre el coste de cada pedido y el tipo de pago.
Hay que comprobar si tenemos duplicados en el dataset y realizar la limpieza de los mismos.

In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

In [2]:
BASE_DIR = Path.cwd().parent
DATA_DIR = (BASE_DIR / "data").resolve()

In [3]:
payments = pd.read_csv(DATA_DIR/"raw"/"olist_order_payments_dataset.csv")
df_payments = pd.DataFrame(payments)

In [4]:
print("\npayments columns:\n", df_payments.columns)


payments columns:
 Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value'],
      dtype='object')


In [5]:
print(df_payments['order_id'].value_counts().max())

29


In [6]:
df_payments.sample(3)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
21767,b8f95bd45718008c382f10f416a3581d,1,credit_card,5,333.36
20005,4f209a7c48b44a32ff3196ddbfb4d67e,1,credit_card,3,156.94
48125,efac22dd2be7399535968466224f162d,1,credit_card,8,241.47


Podemos observar que tenemos hasta 29 duplicados en algunos de los pedidos.

Lo primero que tenemos que hacer es eliminar estos duplicados

In [7]:
payments_total = df_payments.groupby("order_id", as_index=False)["payment_value"].sum()
payments_total.rename(columns={"payment_value": "total_payment"}, inplace=True)

payment_type_main = df_payments.groupby("order_id")["payment_type"].agg(lambda x: x.mode()[0]).reset_index()

max_installments = df_payments.groupby("order_id", as_index=False)["payment_installments"].max()
max_installments.rename(columns={"payment_installments": "max_installments"}, inplace=True)

df_payments = payments_total.merge(payment_type_main, on="order_id", how="left")
df_payments = df_payments.merge(max_installments, on="order_id", how="left")

df_payments.head(3) 

,order_id,total_payment,payment_type,max_installments
0,00010242fe8c5a6d1ba2dd792cb16214,72.19,credit_card,2
1,00018f77f2f0320c557190d7a144bdd3,259.83,credit_card,3
2,000229ec398224ef6ca0657da4fc703e,216.87,credit_card,5


Para eliminar los duplicados en order_id y evitar combinaciones indeseadas (producto cartesiano) al hacer el merge con otras tablas, he realizado la siguiente ingeniería de características:
- *"total_payment"*: Suma de todos los valores de payment_value por pedido, representando el importe total pagado en ese pedido.
- *"payment_type"*: Selección del método de pago más frecuente (moda) dentro de cada pedido.
- *"payment_installments"*: Selección del mayor número de cuotas (payment_installments) registrado para ese pedido.

Con estos agregados, el dataset resultante contiene un solo registro por order_id, evitando duplicados y asegurando un merge limpio y sin producto cartesiano con el resto de tablas.

In [8]:
df_payments.to_csv(DATA_DIR/"processed"/"processed_payments.csv", index=False)